In [15]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np 
import openpyxl
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier  
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re


url = 'https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures'
page = requests.get(url)
html_content = page.content
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table')
df = pd.read_html(str(table))[0].dropna(subset = 'Wk').reset_index(drop=True)
df['Wk'] = df['Wk'].astype(int)

affiches = df[df['Score'].isna()]
first = affiches['Wk'].iloc[0]

df = df[df['Score'].notna()]
df = df[['Wk', 'Home', 'Score', 'Away']]
noms_colonnes = ['Journée','Domicile','Extérieur','Buts domicile','Buts extérieur','Résultat']
df['Buts domicile'] = df['Score'].str[0].astype(int)
df['Buts extérieur'] = df['Score'].str[2].astype(int)
df.drop(['Score'], axis = 1, inplace = True)
df['Résultat'] = -1
df.loc[df['Buts domicile'] > df['Buts extérieur'], 'Résultat'] = 1
df.loc[df['Buts domicile'] == df['Buts extérieur'], 'Résultat'] = 0
df.columns = noms_colonnes

url = "https://www.deux-zero.com/ligue-1/classement-general/edition/2023-2024/init/1/fin/"
df_classement = pd.DataFrame()
annee = re.search(r'\d{4}-\d{4}', url).group()
for i in range(1,first):
    url2 = url + str(i)
    page = requests.get(url2)
    soup = BeautifulSoup(page.text, "html.parser")

    tableau = soup.find("table", class_="tableau")
    equipes = tableau.select('tr.classement td.gras, tr.classement-avec-separateur td.gras')
    equipes_final = []
    for equipe in equipes:
        equipes_final.append(equipe.get_text(strip=True))

    classements = tableau.select('tr.classement span.flag-place, tr.classement-avec-separateur span.flag-place')
    classement_final = []
    for classement in classements:
        if classement.get_text(strip=True) == "-":
            classement_final.append(classement_final[-1])
        else:
            classement_final.append(classement.get_text(strip=True))
    dictionnaire_classement = dict(zip(equipes_final, classement_final))
    if i==1:
        df_classement["Équipes"] = equipes_final
        df_classement["J" + str(i)] = df_classement['Équipes'].map(dictionnaire_classement)
    else:
        df_classement["J" + str(i)] = df_classement['Équipes'].map(dictionnaire_classement)

df_classement.insert(0,"Saison", str(annee))

glossaire = {'AS Monaco FC' : 'Monaco', 'FC Nantes Atlantique' : 'Nantes', 
'Havre AC' : 'Le Havre', 'Montpellier Hérault SC' : 'Montpellier', 'Paris Saint-Germain FC' : 'Paris S-G',  
 'Olympique Lyonnais' : 'Lyon', 'RC Lens' : 'Lens', 'RC Strasbourg' : 'Strasbourg',
 'RC Strasbourg Alsace' : 'Strasbourg', 'OGC Nice' : 'Nice', 
 'Stade Rennais FC' : 'Rennes', 'Olympique de Marseille' : 'Marseille', 'Lille OSC' : 'Lille', 
 'Toulouse FC' : 'Toulouse','FC Metz' : 'Metz', 'FC Lorient' : 'Lorient', 'Stade Brestois 29' : 'Brest',
  'FC Nantes' : 'Nantes', 'Clermont Foot 63' : 'Clermont Foot', 'Stade de Reims' : 'Reims'}

df_classement['Équipes'] = df_classement['Équipes'].replace(glossaire)


df_merge1 = pd.merge(df, df_classement, left_on=['Domicile'], right_on=['Équipes'], how='left')
conditions = [df_merge1['Journée'] == (i + 1) for i in range(1, max(df_merge1['Journée']) + 1)]
valeurs = [df_merge1[f'J{i}'] for i in range(1, max(df_merge1['Journée']) + 1)]
df['Classement D'] = np.select(conditions, valeurs)
df['Classement D'] = df['Classement D'].replace({0: np.nan}).astype(pd.Int64Dtype())



df_merge2 = pd.merge(df, df_classement, left_on=['Extérieur'], right_on=['Équipes'], how='left')
conditions = [df_merge2['Journée'] == (i + 1) for i in range(1, max(df_merge2['Journée']) + 1)]
valeurs = [df_merge2[f'J{i}'] for i in range(1, max(df_merge2['Journée']) + 1)]
df['Classement E'] = np.select(conditions, valeurs)
df['Classement E'] = df['Classement E'].replace({0: np.nan}).astype(pd.Int64Dtype())

df.insert(3, 'Lieu', 'Domicile')
noms_colonnes = ['Journée', 'Equipe 1', 'Equipe 2', 'Lieu', 'Buts Equipe 1', 'Buts Equipe 2', 'Résultat', 'Classement Equipe 1', 'Classement Equipe 2']
df.columns = noms_colonnes
df_copie = df.copy()[['Journée', 'Equipe 2', 'Equipe 1', 'Lieu', 'Buts Equipe 2', 'Buts Equipe 1', 'Résultat', 'Classement Equipe 2', 'Classement Equipe 1']]
noms_colonnes = [ 'Journée', 'Equipe 1', 'Equipe 2', 'Lieu', 'Buts Equipe 1', 'Buts Equipe 2', 'Résultat', 'Classement Equipe 1', 'Classement Equipe 2']
df_copie.columns = noms_colonnes
df_copie['Lieu'] = 'Extérieur'
df = df._append(df_copie, ignore_index=True)
df = df.sort_values(by=['Equipe 1', 'Journée']).reset_index(drop=True)
conditions = [
    (df['Buts Equipe 1'] > df['Buts Equipe 2']),
    (df['Buts Equipe 1'] < df['Buts Equipe 2'])
]
valeurs = [1, -1]
df['Résultat'] = 0
df['Résultat'] = np.select(conditions, valeurs)


df['Moyenne_BM par 1'] = (df.groupby(['Equipe 1'])['Buts Equipe 1'].cumsum() - df['Buts Equipe 1']) / (df.groupby(['Equipe 1'])['Journée'].cumcount())
df['Moyenne_BM par 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BM par 1_y']
df['Moyenne_BM par 1 selon Lieu'] = (df.groupby(['Equipe 1', 'Lieu'])['Buts Equipe 1'].cumsum() - df['Buts Equipe 1']) / (df.groupby(['Equipe 1', 'Lieu'])['Journée'].cumcount())
df['Moyenne_BM par 2 selon Lieu'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BM par 1 selon Lieu_y']
df['Moyenne_BE par 1'] = (df.groupby(['Equipe 1'])['Buts Equipe 2'].cumsum() - df['Buts Equipe 2']) / (df.groupby(['Equipe 1'])['Journée'].cumcount())
df['Moyenne_BE par 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BE par 1_y']
df['Moyenne_BE par 1 selon Lieu'] = (df.groupby(['Equipe 1', 'Lieu'])['Buts Equipe 2'].cumsum() - df['Buts Equipe 2']) / (df.groupby(['Equipe 1', 'Lieu'])['Journée'].cumcount())
df['Moyenne_BE par 2 selon Lieu'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BE par 1 selon Lieu_y']
df['Forme 1'] = df.groupby(['Equipe 1'])['Résultat'].rolling(window=6, min_periods=2).sum().reset_index(drop=True)-df['Résultat']
df['Forme 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Forme 1_y']
df2 = pd.read_csv('C:\\Users\\vtgra\\Desktop\\Projet python\\dataframe_final.csv',encoding = 'utf-8')
df['Historique'] = pd.merge(df,df2.groupby(['Equipe 1', 'Equipe 2'])['Résultat'].sum(), on =[ 'Equipe 1', 'Equipe 2'], how='left')['Résultat_y']
df





,Journée,Equipe 1,Equipe 2,Lieu,Buts Equipe 1,Buts Equipe 2,Résultat,Classement Equipe 1,Classement Equipe 2,Moyenne_BM par 1,Moyenne_BM par 2,Moyenne_BM par 1 selon Lieu,Moyenne_BM par 2 selon Lieu,Moyenne_BE par 1,Moyenne_BE par 2,Moyenne_BE par 1 selon Lieu,Moyenne_BE par 2 selon Lieu,Forme 1,Forme 2,Historique
0,1,Brest,Lens,Domicile,3,2,1,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2,Brest,Le Havre,Extérieur,2,1,1,3,7,3.000000,2.000000,NaN,NaN,2.000000,2.000000,NaN,NaN,1.0,0.0,NaN
2,3,Brest,Marseille,Extérieur,0,2,-1,2,6,2.500000,2.000000,2.000000,2.0,1.500000,1.500000,1.000000,1.000000,2.0,1.0,-4.0
3,4,Brest,Rennes,Domicile,0,0,0,4,6,1.666667,2.666667,3.000000,1.0,1.666667,1.333333,2.000000,1.000000,1.0,1.0,-7.0
4,5,Brest,Reims,Extérieur,2,1,1,5,4,1.250000,2.000000,1.000000,2.0,1.250000,1.250000,1.500000,0.000000,1.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,11,Toulouse,Le Havre,Domicile,1,2,-1,12,13,1.100000,1.000000,1.750000,0.8,1.300000,1.200000,1.000000,1.400000,-1.0,-1.0,2.0
266,12,Toulouse,Lille,Extérieur,1,1,0,14,5,1.090909,1.272727,0.666667,1.4,1.363636,0.909091,1.500000,0.400000,-1.0,3.0,-5.0
267,13,Toulouse,Nice,Extérieur,0,1,-1,15,2,1.083333,1.083333,0.714286,1.0,1.333333,0.333333,1.428571,0.166667,-2.0,4.0,-7.0
268,14,Toulouse,Lorient,Domicile,1,1,0,15,16,1.000000,1.307692,1.600000,1.0,1.307692,1.846154,1.200000,2.166667,-3.0,-2.0,0.0


In [16]:

def fonction_journées(event):
    journées = int(journées_combobox.get()[1:])

    label = tk.Label(window, text="Choisissez un match:")
    label.pack(pady=10)

    matchs = [affiche for affiche, semaine in zip(affiches['Home'] + ' - ' + affiches['Away'], affiches['Wk']) if semaine == journées]



    matchs_combobox = ttk.Combobox(window, values=matchs, state="readonly")
    matchs_combobox.set("Sélectionnez un match")
    matchs_combobox.bind("<<ComboboxSelected>>", fonction_stats)
    matchs_combobox.pack(padx=10, pady=10)


def fonction_stats(event):
    return 0

window = tk.Tk()
window.title("Sélection de la Journée")
window.attributes('-fullscreen', True)


label = tk.Label(window, text="Choisissez une journée:")
label.pack(pady=10)



journées = ['J' + str(i) for i in range(first,39)]

journées_combobox = ttk.Combobox(window, values=journées, state="readonly")
journées_combobox.set("Sélectionnez une journée")
journées_combobox.bind("<<ComboboxSelected>>", fonction_journées)
journées_combobox.pack(padx=10, pady=10)

window.mainloop()